# COMP 432 - BIXI

In [1]:
import numpy as np
import pandas as pd
from zipfile import ZipFile
from tqdm import tqdm
import time
# Removing scientific notation
pd.set_option('display.float_format', lambda x: '%.3f' % x)

## Load BIXI data

In [2]:
zf = ZipFile('./data/bixi/biximontrealrentals2019-33ea73.zip')
bixi_2019_04 = pd.read_csv(zf.open('OD_2019-04.csv'), parse_dates=['start_date', 'end_date'])
bixi_2019_05 = pd.read_csv(zf.open('OD_2019-05.csv'), parse_dates=['start_date', 'end_date'])
bixi_2019_06 = pd.read_csv(zf.open('OD_2019-06.csv'), parse_dates=['start_date', 'end_date'])
bixi_2019_07 = pd.read_csv(zf.open('OD_2019-07.csv'), parse_dates=['start_date', 'end_date'])
bixi_2019_08 = pd.read_csv(zf.open('OD_2019-08.csv'), parse_dates=['start_date', 'end_date'])
bixi_2019_09 = pd.read_csv(zf.open('OD_2019-09.csv'), parse_dates=['start_date', 'end_date'])
bixi_2019_10 = pd.read_csv(zf.open('OD_2019-10.csv'), parse_dates=['start_date', 'end_date'])
bixi_2019_stations = pd.read_csv(zf.open('Stations_2019.csv'), dtype={'name': 'str'})

C:\Users\Simon\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (1,3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
bixi_2019_04.head()

,start_date,start_station_code,end_date,end_station_code,duration_sec,is_member
0,2019-04-14 07:55:22,6001,2019-04-14 08:07:16,6132,713,1
1,2019-04-14 07:59:31,6411,2019-04-14 08:09:18,6411,587,1
2,2019-04-14 07:59:55,6097,2019-04-14 08:12:11,6036,736,1
3,2019-04-14 07:59:57,6310,2019-04-14 08:27:58,6345,1680,1
4,2019-04-14 08:00:37,7029,2019-04-14 08:14:12,6250,814,0


In [4]:
bixi_2019_stations.head()

,Code,name,latitude,longitude
0,10002,Métro Charlevoix (Centre / Charlevoix),45.478,-73.570
1,4000,Jeanne-d'Arc / Ontario,45.550,-73.542
2,4001,Graham / Brookfield,45.520,-73.630
3,4002,Graham / Wicksteed,45.517,-73.640
4,5002,St-Charles / Montarville,45.534,-73.515


## Load weather data

In [5]:
# Note: Use Date/Time column instead of Time column
weather_2019_04 = pd.read_csv('./data/weather/2019/en_climate_hourly_QC_7025251_04-2019_P1H.csv', parse_dates=['Date/Time'], dtype={'Station Name': 'str', 'Weather': 'str'})
weather_2019_05 = pd.read_csv('./data/weather/2019/en_climate_hourly_QC_7025251_05-2019_P1H.csv', parse_dates=['Date/Time'], dtype={'Station Name': 'str', 'Weather': 'str'})
weather_2019_06 = pd.read_csv('./data/weather/2019/en_climate_hourly_QC_7025251_06-2019_P1H.csv', parse_dates=['Date/Time'], dtype={'Station Name': 'str', 'Weather': 'str'})
weather_2019_07 = pd.read_csv('./data/weather/2019/en_climate_hourly_QC_7025251_07-2019_P1H.csv', parse_dates=['Date/Time'], dtype={'Station Name': 'str', 'Weather': 'str'})
weather_2019_08 = pd.read_csv('./data/weather/2019/en_climate_hourly_QC_7025251_08-2019_P1H.csv', parse_dates=['Date/Time'], dtype={'Station Name': 'str', 'Weather': 'str'})
weather_2019_09 = pd.read_csv('./data/weather/2019/en_climate_hourly_QC_7025251_09-2019_P1H.csv', parse_dates=['Date/Time'], dtype={'Station Name': 'str', 'Weather': 'str'})
weather_2019_10 = pd.read_csv('./data/weather/2019/en_climate_hourly_QC_7025251_10-2019_P1H.csv', parse_dates=['Date/Time'], dtype={'Station Name': 'str', 'Weather': 'str'})

In [6]:
weather_2019_04.head()

,Longitude (x),Latitude (y),Station Name,Climate ID,Date/Time,Year,Month,Day,Time,Temp (°C),...,Wind Spd Flag,Visibility (km),Visibility Flag,Stn Press (kPa),Stn Press Flag,Hmdx,Hmdx Flag,Wind Chill,Wind Chill Flag,Weather
0,-73.740,45.470,MONTREAL INTL A,7025251,2019-04-01 00:00:00,2019,4,1,00:00,-1.500,...,nan,24.100,nan,100.750,nan,nan,nan,-7.000,nan,Snow
1,-73.740,45.470,MONTREAL INTL A,7025251,2019-04-01 01:00:00,2019,4,1,01:00,-1.700,...,nan,24.100,nan,100.790,nan,nan,nan,-9.000,nan,Cloudy
2,-73.740,45.470,MONTREAL INTL A,7025251,2019-04-01 02:00:00,2019,4,1,02:00,-2.900,...,nan,24.100,nan,100.860,nan,nan,nan,-9.000,nan,NaN
3,-73.740,45.470,MONTREAL INTL A,7025251,2019-04-01 03:00:00,2019,4,1,03:00,-3.900,...,nan,24.100,nan,100.920,nan,nan,nan,-10.000,nan,NaN
4,-73.740,45.470,MONTREAL INTL A,7025251,2019-04-01 04:00:00,2019,4,1,04:00,-4.800,...,nan,24.100,nan,101.030,nan,nan,nan,-11.000,nan,Clear


## Merging all data together

### BIXI data

Merging all 2019 bixi trips together

In [7]:
bixi_2019_all = pd.concat([bixi_2019_04, bixi_2019_05, bixi_2019_06, bixi_2019_07, bixi_2019_08, bixi_2019_09, bixi_2019_10])
bixi_2019_all.sample(5)

,start_date,start_station_code,end_date,end_station_code,duration_sec,is_member
916916,2019-08-27 17:34:27,7057,2019-08-27 18:03:19,6016,1731,1
295817,2019-05-13 18:15:16,6181,2019-05-13 18:23:41,6034,505,1
529144,2019-07-16 07:32:58,6706,2019-07-16 07:47:14,6727,855,1
39021,2019-10-02 21:23:48,7024,2019-10-02 21:29:01,6161,312,1
370395,2019-09-13 17:32:46,6064,2019-09-13 17:49:49,7073,1023,1


Append station data to all Bixi Trips

***May take a while to run**

In [8]:
bixi_2019_all_with_stations = bixi_2019_all.merge(bixi_2019_stations, left_index=True, left_on='start_station_code', right_on='Code').drop('Code', 1).rename({'name': 'start_name', 'latitude': 'start_latitude', 'longitude': 'start_longitude'}, axis=1).merge(bixi_2019_stations, left_index=True, left_on='end_station_code', right_on='Code').drop('Code', 1).rename({'name': 'end_name', 'latitude': 'end_latitude', 'longitude': 'end_longitude'}, axis=1)
bixi_2019_all_with_stations.sample(5)

,start_date,start_station_code,end_date,end_station_code,duration_sec,is_member,start_name,start_latitude,start_longitude,end_name,end_latitude,end_longitude
287,2019-07-23 10:15:45,6267,2019-07-23 10:34:03,6322,1098,1,Chabot / Beaubien,45.544,-73.596,St-Dominique / Gounod,45.539,-73.628
364,2019-10-18 13:21:35,7054,2019-10-18 13:29:33,6414,477,1,de la Côte St-Paul / St-Ambroise,45.468,-73.594,Parc St-Henri (Laporte / Guay),45.480,-73.586
331,2019-05-02 10:07:09,6744,2019-05-02 10:21:25,6375,856,1,Hamilton / Jolicoeur,45.456,-73.598,Métro Place St-Henri (St-Ferdinand / St-Jacques),45.477,-73.587
200,2019-06-13 23:48:04,6138,2019-06-14 00:10:54,6206,1369,1,Gauthier / de Lorimier,45.532,-73.565,Prince-Arthur / du Parc,45.511,-73.575
123,2019-10-29 18:34:20,6120,2019-10-29 18:37:49,6126,209,1,Métro Frontenac (Ontario / du Havre),45.534,-73.552,Fullum / de Rouen,45.532,-73.559


### Weather

Merging weather data

In [9]:
weather_2019_all = pd.concat([weather_2019_04, weather_2019_05, weather_2019_06, weather_2019_07, weather_2019_08, weather_2019_09, weather_2019_10])
weather_2019_all.sample(10)

,Longitude (x),Latitude (y),Station Name,Climate ID,Date/Time,Year,Month,Day,Time,Temp (°C),...,Wind Spd Flag,Visibility (km),Visibility Flag,Stn Press (kPa),Stn Press Flag,Hmdx,Hmdx Flag,Wind Chill,Wind Chill Flag,Weather
38,-73.740,45.470,MONTREAL INTL A,7025251,2019-10-02 14:00:00,2019,10,2,14:00,12.100,...,nan,24.100,nan,101.140,nan,nan,nan,nan,nan,NaN
323,-73.740,45.470,MONTREAL INTL A,7025251,2019-08-14 11:00:00,2019,8,14,11:00,20.800,...,nan,48.300,nan,100.940,nan,nan,nan,nan,nan,NaN
394,-73.740,45.470,MONTREAL INTL A,7025251,2019-04-17 10:00:00,2019,4,17,10:00,9.000,...,nan,48.300,nan,101.820,nan,nan,nan,nan,nan,Clear
395,-73.740,45.470,MONTREAL INTL A,7025251,2019-10-17 11:00:00,2019,10,17,11:00,7.600,...,nan,9.700,nan,98.330,nan,nan,nan,nan,nan,"Rain,Fog"
436,-73.740,45.470,MONTREAL INTL A,7025251,2019-09-19 04:00:00,2019,9,19,04:00,7.900,...,nan,24.100,nan,102.240,nan,nan,nan,nan,nan,Clear
214,-73.740,45.470,MONTREAL INTL A,7025251,2019-07-09 22:00:00,2019,7,9,22:00,22.800,...,nan,24.100,nan,101.150,nan,26.000,nan,nan,nan,Mainly Clear
363,-73.740,45.470,MONTREAL INTL A,7025251,2019-10-16 03:00:00,2019,10,16,03:00,8.300,...,nan,24.100,nan,101.050,nan,nan,nan,nan,nan,NaN
210,-73.740,45.470,MONTREAL INTL A,7025251,2019-04-09 18:00:00,2019,4,9,18:00,-1.000,...,nan,12.900,nan,99.790,nan,nan,nan,-7.000,nan,Snow
282,-73.740,45.470,MONTREAL INTL A,7025251,2019-09-12 18:00:00,2019,9,12,18:00,15.800,...,nan,48.300,nan,102.220,nan,nan,nan,nan,nan,NaN
233,-73.740,45.470,MONTREAL INTL A,7025251,2019-06-10 17:00:00,2019,6,10,17:00,27.500,...,nan,48.300,nan,100.770,nan,29.000,nan,nan,nan,NaN


### BIXI + Weather

Merging weather data (Tempeture and weather) with BIXI data

In [10]:
bixi_2019_all_with_stations_and_weather = bixi_2019_all_with_stations.merge(weather_2019_all[['Date/Time', 'Weather', 'Temp (°C)']], left_on=bixi_2019_all_with_stations['start_date'].dt.floor('H'), right_on='Date/Time').drop('Date/Time', 1).rename({'Weather': 'start_weather', 'Temp (°C)': 'start_temp'}, axis=1)
bixi_2019_all_with_stations_and_weather.sample(10)

,start_date,start_station_code,end_date,end_station_code,duration_sec,is_member,start_name,start_latitude,start_longitude,end_name,end_latitude,end_longitude,start_weather,start_temp
2282680,2019-08-10 23:42:34,6170,2019-08-10 23:56:38,7083,843,0,de Mentana / Laurier,45.530,-73.588,Parc de Bullion (de Bullion / Prince-Arthur),45.516,-73.572,NaN,15.400
1586567,2019-09-27 16:29:12,7041,2019-09-27 16:47:57,6080,1124,1,de Maisonneuve / Peel,45.501,-73.575,Métro Vendôme (de Marlowe / de Maisonneuve),45.474,-73.605,Mainly Clear,19.300
3008492,2019-06-18 14:24:52,6421,2019-06-18 14:30:36,6919,343,1,Cartier / Rosemont,45.541,-73.591,Bibliothèque de Rosemont (9e avenue / Rosemont),45.552,-73.582,NaN,25.600
3349872,2019-05-28 08:08:48,6123,2019-05-28 08:24:53,6041,965,1,Alexandre-DeSève / la Fontaine,45.523,-73.558,St-Jacques / McGill,45.501,-73.560,NaN,11.000
3260105,2019-08-09 18:54:27,7075,2019-08-09 18:59:50,7016,322,1,CIUSSS (du Trianon / Pierre-de-Coubertin),45.588,-73.532,Métro Langelier (Langelier / Sherbrooke),45.583,-73.542,NaN,21.400
4958009,2019-10-02 13:25:07,7043,2019-10-02 13:33:37,6396,510,0,Ernest-Gendreau / du Mont-Royal,45.550,-73.564,Métro Pie-IX (Pierre-de-Coubertin / Pie-IX),45.554,-73.552,Mostly Cloudy,12.800
186918,2019-06-10 17:43:17,6385,2019-06-10 17:53:39,6929,621,1,de Bordeaux / Gilford,45.536,-73.576,St-André / St-Grégoire,45.531,-73.592,NaN,27.500
4047535,2019-10-29 16:35:53,6389,2019-10-29 16:42:25,6343,392,1,Boyer / Jean-Talon,45.542,-73.612,Marquette / Villeray,45.551,-73.615,Mostly Cloudy,15.000
90749,2019-06-10 16:01:43,6064,2019-06-10 16:22:59,6080,1276,1,Métro Peel (de Maisonneuve / Stanley),45.500,-73.575,Métro Vendôme (de Marlowe / de Maisonneuve),45.474,-73.605,Mostly Cloudy,28.200
3345540,2019-05-24 08:31:08,6358,2019-05-24 08:57:38,6359,1589,1,1ère avenue / St-Zotique,45.552,-73.594,Ste-Catherine / St-Laurent,45.510,-73.564,Rain,13.000


## Data Cleaning

### Checking if there are missing data
Only the weather data is missing some values

In [11]:
bixi_2019_all_with_stations_and_weather.isnull().sum()

start_date                  0
start_station_code          0
end_date                    0
end_station_code            0
duration_sec                0
is_member                   0
start_name                  0
start_latitude              0
start_longitude             0
end_name                    0
end_latitude                0
end_longitude               0
start_weather         3459073
start_temp               1494
dtype: int64

### Checking the contents of the "Weather" column
Some of the weather values contains more than 1 condition. We'll have to take this into account while training.

In [12]:
bixi_2019_all_with_stations_and_weather['start_weather'].value_counts()

Mostly Cloudy                              633845
Mainly Clear                               400336
Cloudy                                     304650
Clear                                      281318
Rain                                       165817
Rain Showers                               102038
Rain,Fog                                    20620
Drizzle,Fog                                 17388
Fog                                         16463
Drizzle                                     16140
Thunderstorms,Rain Showers                  11358
Moderate Rain,Fog                            4062
Moderate Rain Showers                        3378
Rain,Rain Showers                            1437
Moderate Rain                                1202
Thunderstorms,Rain                            793
Thunderstorms,Moderate Rain Showers,Fog       635
Thunderstorms,Moderate Rain Showers           612
Thunderstorms,Heavy Rain Showers              577
Thunderstorms                                 274


We will have to find a way to split the weather conditions into smaller parts. 

The best way to do so is to create new features to the data that represent the weather and set the flag to either 0 or 1

Here are the unique weather conditions:
- Mostly Cloudy
- Mainly Clear
- Cloudy
- Clear
- Rain
- Rain Showers
- Fog
- Drizzle
- Moderate Rain Showers
- Moderate Rain
- Heavy Rain Showers
- Thunderstorms

Also, more than half of the data does not have a weather appended to it. I'm not sure if this will affect the results or not. 

## Expanding data

### Weather flags
First, we create new features for the weather data and set the appropriate flag for all trip data

***May take a while to run**

In [13]:
# Using "temp" variable so I can display the results and debug before overwriting the original variable
temp = bixi_2019_all_with_stations_and_weather

temp = temp.join(temp['start_weather'].str.get_dummies(','))

bixi_2019_all_with_stations_and_weather = temp

bixi_2019_all_with_stations_and_weather.sample(5)

,start_date,start_station_code,end_date,end_station_code,duration_sec,is_member,start_name,start_latitude,start_longitude,end_name,...,Drizzle,Fog,Heavy Rain Showers,Mainly Clear,Moderate Rain,Moderate Rain Showers,Mostly Cloudy,Rain,Rain Showers,Thunderstorms
270112,2019-05-06 13:30:06,6302,2019-05-06 13:49:19,7069,1153,0,Stuart / St-Viateur,45.516,-73.608,Jeanne-Mance / René-Lévesque,...,0,0,0,0,0,0,1,0,0,0
4137445,2019-05-19 18:24:20,6205,2019-05-19 18:27:18,7033,177,1,Milton / du Parc,45.509,-73.574,Aylmer / Prince-Arthur,...,0,0,0,0,0,0,0,0,1,0
1543654,2019-05-27 09:24:58,7052,2019-05-27 09:35:57,7060,659,1,Shearer / Centre,45.483,-73.562,de l'Église / de Verdun,...,0,0,0,0,0,0,0,1,0,0
2480875,2019-10-21 18:19:25,6027,2019-10-21 18:48:35,6139,1749,1,de Maisonneuve / Mansfield (sud),45.502,-73.573,Gauthier / Parthenais,...,0,0,0,0,0,0,0,0,0,0
3575602,2019-07-03 22:46:28,6148,2019-07-03 22:57:33,6163,664,1,Émile-Duployé / Sherbrooke,45.527,-73.565,Marquette / Laurier,...,0,0,0,0,0,0,0,0,0,0


### Splitting start_date to year, month, day, hour, minute, second columns
Also adding seconds since midnight for simpler normalization

In [14]:
temp = bixi_2019_all_with_stations_and_weather

temp['start_year'] = temp['start_date'].dt.year
temp['start_month'] = temp['start_date'].dt.month
temp['start_day'] = temp['start_date'].dt.day
temp['start_hour'] = temp['start_date'].dt.hour
temp['start_minute'] = temp['start_date'].dt.minute
temp['start_second'] = temp['start_date'].dt.second
temp['start_seconds_since_midnight'] = (temp['start_date'] - pd.to_datetime(temp['start_date'].dt.strftime('%Y-%m-%d') + ' ' + '00:00:00')).astype('timedelta64[s]')

bixi_2019_all_with_stations_and_weather = temp

bixi_2019_all_with_stations_and_weather.sample(5)

,start_date,start_station_code,end_date,end_station_code,duration_sec,is_member,start_name,start_latitude,start_longitude,end_name,...,Rain,Rain Showers,Thunderstorms,start_year,start_month,start_day,start_hour,start_minute,start_second,start_seconds_since_midnight
2609654,2019-08-01 13:04:19,6388,2019-08-01 13:14:02,6131,583,1,d'Orléans / Hochelaga,45.551,-73.549,Fullum / Sherbrooke,...,0,0,0,2019,8,1,13,4,19,47059.000
2060028,2019-09-03 08:45:43,6136,2019-09-03 08:53:54,6211,490,1,Métro Laurier (Rivard / Laurier),45.528,-73.589,Roy / St-Laurent,...,0,0,0,2019,9,3,8,45,43,31543.000
1272855,2019-04-17 16:53:31,6042,2019-04-17 17:29:15,7030,2143,1,de la Gauchetière / Robert-Bourassa,45.501,-73.565,de Bordeaux / Marie-Anne,...,0,0,0,2019,4,17,16,53,31,60811.000
3473433,2019-04-17 20:20:07,6413,2019-04-17 20:26:55,7047,408,1,Union / Cathcart,45.504,-73.569,Ottawa / Peel,...,0,0,0,2019,4,17,20,20,7,73207.000
798665,2019-09-27 17:53:01,6731,2019-09-27 18:12:41,6393,1179,1,28e avenue / Rosemont,45.564,-73.571,Dézéry / Ontario,...,0,0,0,2019,9,27,17,53,1,64381.000


### Adding number of trips in a +/- 30 minutes time frame of that particular station

This will serve as our "y" parameter, aka the value we are trying to predict

In [15]:
# _m = bixi_2019_all_with_stations_and_weather['start_month'] == 8
# _d = bixi_2019_all_with_stations_and_weather['start_day'] == 14
# _h = bixi_2019_all_with_stations_and_weather['start_hour'] == 8
# _c = bixi_2019_all_with_stations_and_weather['start_station_code'] == 6176
# _c2 = bixi_2019_all_with_stations_and_weather['start_station_code'] == 6014

# temp = bixi_2019_all_with_stations_and_weather[_m & _d & (_c | _c2)].sample(10)

#2019-09-22 15:29:16 	6152
#2019-04-14 08:49:42 	6176
        
temp = bixi_2019_all_with_stations_and_weather
len(temp)

5442288

Slow. 500+ hours to run

In [16]:
# # I couldn't figure out how to vectorize this...

# with tqdm(total=len(temp)) as pbar: # Progress bar
#     for i, row in temp.iterrows():
#         # Get the current row's station code
#         target_station_code = row['start_station_code']
#         target_start_date = row['start_date']

#         # Retrieve all the data for that particular station code
#         df_station = temp[(temp['start_station_code'] == target_station_code)]
        
#         # Count how many trips were started +/- 30min of the start time
#         sum_count = df_station['start_date'].between(
#             target_start_date + pd.Timedelta('-30 minutes'),
#             target_start_date + pd.Timedelta('30 minutes')
#         ).sum()

#         #Assign sum to new trip_count column
#         temp.loc[i,'trip_count'] = sum_count
        
#         #Update progress bar
#         pbar.update(1)

# temp.to_pickle('./data/bixi_2019_all_with_stations_and_weather_and_trip_count.pkl')

In [17]:
# processed = pd.read_pickle('./data/bixi_2019_all_with_stations_and_weather_and_trip_count.pkl')
# processed.sort_values(by='start_date')

A little bit better but still slow. Maybe 250 hours?

In [18]:
# def count_trips(row):
#     station = temp['start_station_code'] == row['start_station_code']
#     t_date = row['start_date']
#     t_start = row['start_date'] + pd.Timedelta('-30 minutes')
#     t_end = row['start_date'] + pd.Timedelta('30 minutes')
#     return len(temp[station & (t_start < t_date) & (t_date < t_end)])

In [19]:
# start_time = time.time()
# temp['trip_count'] = temp.apply(count_trips, axis=1)
# print("--- %s seconds ---" % (time.time() - start_time))
# temp.to_pickle('./data/bixi_2019_all_with_stations_and_weather_and_trip_count.pkl')

# temp.sort_values(by='start_date')

In [20]:
# processed = pd.read_pickle('./data/bixi_2019_all_with_stations_and_weather_and_trip_count.pkl')
# processed.sort_values(by='start_date')

Better - I think we can optimize it even more but that's good enough. Run time ~ 1 hour

In [21]:
# group = temp.groupby('start_station_code')

# with tqdm(total=len(group)) as pbar: # Progress bar
#     trip_count_temp = []
#     for code, df in group:
#         trip_count_group = df.apply(lambda x: 
#                                     df['start_date'].between(
#                                         x['start_date'] + pd.Timedelta('-30 minutes'),
#                                         x['start_date'] + pd.Timedelta('30 minutes')).sum(), axis=1)
#         trip_count_temp.append(trip_count_group)
#         pbar.update(1)

# print("Saving trip_count_temp to temp file...")
# np.save('./data/trip_count_temp.npy', trip_count_temp)
# print("Done")


# # DO NOT RUN BELOW IF PROCESSING ALL THE DATA. IT IS LIKELY TO THROW AN OUT OF MEMORY ERROR


# print("Converting... This may take a while but it should be faster than the previous one")
# # Throws a out of memory error if processing 5 million + data with 32GB ram
# trip_count = pd.DataFrame(pd.DataFrame(trip_count_temp).T.apply(lambda x: x.sum(), axis=1), columns=['trip_count'])
# print("Done")

# trip_count

# print("Concat to temp...")
# tc = pd.concat([temp, trip_count], axis=1, sort=False)
# print("Done")

# print("Saving to disk...")
# tc.to_pickle('./data/bixi_2019_all_with_stations_and_weather_and_trip_count.pkl')
# print("Done")

In [22]:
trip_count_temp_file = np.load('./data/trip_count_temp.npy', allow_pickle=True)

Merging series together

In [23]:
trip_count_series = pd.Series()

with tqdm(total=len(trip_count_temp_file)) as pbar: # Progress bar
    for series in trip_count_temp_file:
        trip_count_series = trip_count_series.append(series)
        pbar.update(1)

print(trip_count_series.shape)
trip_count_series = trip_count_series.sort_index()
trip_count_series.describe()

100%|██████████| 619/619 [00:06<00:00, 55.77it/s] 


(5442288,)


count   5442288.000
mean          7.828
std           8.266
min           1.000
25%           3.000
50%           6.000
75%          10.000
max         223.000
dtype: float64

Merge trip count data to overall data

In [24]:
temp = pd.concat([temp, trip_count_series], axis=1, sort=False)

In [25]:
temp = temp.rename({0: "trip_count"}, axis=1)

In [26]:
temp.sample(5)

,start_date,start_station_code,end_date,end_station_code,duration_sec,is_member,start_name,start_latitude,start_longitude,end_name,...,Rain Showers,Thunderstorms,start_year,start_month,start_day,start_hour,start_minute,start_second,start_seconds_since_midnight,trip_count
2267349,2019-05-07 19:22:07,6261,2019-05-07 19:39:03,7009,1015,1,Louis-Hémon / Rosemont,45.545,-73.588,Place Émilie-Gamelin (de Maisonneuve / Berri),...,0,0,2019,5,7,19,22,7,69727.000,2
4400773,2019-06-14 22:43:36,6268,2019-06-14 22:50:09,6272,392,1,Chambord / Beaubien,45.540,-73.599,de Bordeaux / St-Zotique,...,0,0,2019,6,14,22,43,36,81816.000,5
3179627,2019-06-04 20:45:02,6919,2019-06-04 20:47:01,6082,118,1,Bibliothèque de Rosemont (9e avenue / Rosemont),45.552,-73.582,5e avenue / Rosemont,...,0,0,2019,6,4,20,45,2,74702.000,3
367358,2019-06-03 07:46:09,6433,2019-06-03 07:57:13,6027,664,1,Hillside / Ste-Catherine,45.483,-73.591,de Maisonneuve / Mansfield (sud),...,0,0,2019,6,3,7,46,9,27969.000,7
1431073,2019-08-13 18:25:15,6040,2019-08-13 18:35:00,6099,585,1,St-Jacques / St-Pierre,45.503,-73.559,Ste-Catherine / Labelle,...,0,0,2019,8,13,18,25,15,66315.000,17
